In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [21]:
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/transition/data/smartphone+based+recognition+of+human+activities+and+postural+transitions/RawData'
OUTPUT_DIR = '/content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/transition/results/HAPT'
TODAY_DATE = "20251206"

In [18]:
TRANSITION_LABELS = {
    7: 'STAND_TO_SIT',
    8: 'SIT_TO_STAND',
    9: 'SIT_TO_LIE',
    10: 'LIE_TO_SIT',
    11: 'STAND_TO_LIE',
    12: 'LIE_TO_STAND'
}

In [19]:
def visualize_hapt_transitions(data_dir):
    labels_path = os.path.join(data_dir, 'labels.txt')

    if not os.path.exists(labels_path):
        print(f"Error: 'labels.txt' 파일을 찾을 수 없음: {data_dir}")
        return

    # 1. labels.txt 읽기
    print("Loading labels.txt...")
    try:
        labels_df = pd.read_csv(labels_path, header=None, sep=' ', names=['exp_id', 'user_id', 'act_id', 'start', 'end'])
    except Exception as e:
        print(f"Error reading labels.txt: {e}")
        return

    # 2. 전이 구간(7~12번)만 필터링
    transition_df = labels_df[labels_df['act_id'].isin(TRANSITION_LABELS.keys())].copy()

    if transition_df.empty:
        print("전이 데이터 없음")
        return

    # 3. 실험 순서대로 정렬 (User -> Experiment -> Start Time 순)
    transition_df = transition_df.sort_values(by=['user_id', 'exp_id', 'start'])

    print(f"Found total {len(transition_df)} transition segments. Processing all...")

    # 4. 모든 구간에 대해 루프 (iterrows 사용)
    for idx, row in transition_df.iterrows():
        user = int(row['user_id'])
        exp_id = int(row['exp_id'])
        act_id = int(row['act_id'])
        start = int(row['start'])
        end = int(row['end'])

        act_name = TRANSITION_LABELS[act_id]

        # Raw Data 파일명 구성 (acc_expXX_userYY.txt)
        file_name = f"acc_exp{exp_id:02d}_user{user:02d}.txt"
        file_path = os.path.join(data_dir, file_name)

        if not os.path.exists(file_path):
            # print(f"   File not found: {file_name}")
            continue

        try:
            # 데이터 읽기
            raw_data = pd.read_csv(file_path, header=None, sep=' ', names=['ax', 'ay', 'az'])

            # 구간 자르기 (Slicing)
            segment = raw_data.iloc[start:end+1]

            # --- 그래프 그리기 ---
            plt.figure(figsize=(10, 3))

            # 3축 가속도
            plt.plot(segment.index, segment['ax'], label='Acc X', color='r', alpha=0.6, linewidth=0.8)
            plt.plot(segment.index, segment['ay'], label='Acc Y', color='g', alpha=0.6, linewidth=0.8)
            plt.plot(segment.index, segment['az'], label='Acc Z', color='b', alpha=0.6, linewidth=0.8)

            # Magnitude (벡터 크기)
            mag = np.sqrt(segment['ax']**2 + segment['ay']**2 + segment['az']**2)
            plt.plot(segment.index, mag, label='Magnitude', color='black', linestyle='--', linewidth=1.5)

            plt.title(f"User {user} - {act_name} (Exp {exp_id}) [Index: {start}-{end}]")
            plt.xlabel('Sample Index')
            plt.ylabel('Acceleration (g)')
            plt.legend(loc='upper right', fontsize='small')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()

            # --- 파일 저장 ---
            # 파일명: HAPT_User01_Exp01_STAND_TO_SIT_Start100_20251206.png
            save_filename = f"HAPT_User{user:02d}_Exp{exp_id:02d}_{act_name}_Start{start}_{TODAY_DATE}.png"
            save_path = os.path.join(OUTPUT_DIR, save_filename)

            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"   💾 Saved: {save_filename}")

            plt.show() # 스크롤 압박이 심하면 주석 처리!
            plt.close()

        except Exception as e:
            print(f"   Error processing {file_name} at line {idx}: {e}")

    print("\nAll transition visualizations complete!")

In [22]:
if __name__ == "__main__":
    visualize_hapt_transitions(DATA_DIR)

Output hidden; open in https://colab.research.google.com to view.